# Ethanolamine and glycerol co-metabolism in *Escherichia coli*

Genome-scale modeling of ethanolamine and glycerol co-metabolism in *Escherichia coli*.

This script is provided as supplementary information of Jallet *et al.* (2024).

## Set environment

In [1]:
import cobra
import json
import os
import pandas
import re
from cobra.flux_analysis import flux_variability_analysis

## Load model iDJ1518

In [2]:
model = cobra.io.load_json_model('./model/iDJ1518.json')

Show model summary.

In [3]:
model

Name,iDJ1518
Memory address,194653c0bf0
Number of metabolites,1885
Number of reactions,2721
Number of genes,1518
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, periplasm, extracellular space, Eut BMC"


## Set constraints

To suppress the substrate uptake rates that are present by default in the model and ensure that the pre-defined cytosolic EA-ammonia lyase reaction from the model has a null flux (i.e. force EA to enter into the Eut BMC).

In [5]:
# Set the glucose uptake to null since there is no glucose in the medium
glc_exchange = model.reactions.get_by_id('EX_glc__D_e')
glc_exchange.lower_bound = 0
glc_exchange.upper_bound = 0

# Block ammonia uptake and allow ammonia production
ammo_exchange = model.reactions.get_by_id('EX_nh4_e')
ammo_exchange.lower_bound = 0
ammo_exchange.upper_bound = 1000

# Force EA into the Eut BMC lumen by applying a null flux to the present cytosolic EA-ammonia lyase reaction from the model
ETHAAL_c = model.reactions.get_by_id('ETHAAL')
ETHAAL_c.lower_bound = 0
ETHAAL_c.upper_bound = 0

Constrain substrate uptake rates based on experimental values.

In [6]:
# Set the glyc uptake rate; glycerol represents the primary C source here
glyc_exchange = model.reactions.get_by_id('EX_glyc_e')
glyc_exchange.lower_bound = -14.55
glyc_exchange.upper_bound = -14.55

# Set the EA uptake rate; ethanolamine represents the primary N source and a secondary C source here
etha_exchange = model.reactions.get_by_id('EX_etha_e')
etha_exchange.lower_bound = -7.75
etha_exchange.upper_bound = -7.75

Constrain product excretion rates based on experimental values.

In [7]:
# Ethanol (etoh) generally represents one of the main excereted products as detected by NMR
etoh_exchange = model.reactions.get_by_id('EX_etoh_e')
etoh_exchange.lower_bound = 2.57
etoh_exchange.upper_bound = 2.57

# Acetate (ace) generally represents one of the main excereted products as detected by NMR
ace_exchange = model.reactions.get_by_id('EX_ac_e')
ace_exchange.lower_bound = 1.66
ace_exchange.upper_bound = 1.66

## Optimality analysis

Define the objective function (growth) and determines the optimal growth rate.

In [8]:
# Here by default, aim at maximizing the biomass objective function
model.objective = 'BIOMASS_Ec_iML1515_core_75p37M'
model.objective.expression

# Solve the rates and creates a csv file to store the values
solution = model.optimize()
print('Satus: ', solution.status)
print('Growth rate: %.2f' % solution.objective_value)
solution.fluxes.to_csv("./results/BiomassOpt.csv")

Satus:  optimal
Growth rate: 0.72


Constrain growth rate to experimental value. 

In [9]:
# Set boundaries for the biomass function based on measured growth rates
BIOMASS_accu = model.reactions.get_by_id('BIOMASS_Ec_iML1515_core_75p37M')
BIOMASS_accu.lower_bound = 0.45
BIOMASS_accu.upper_bound = 0.45

Change the objective function to maximize ATP production.

In [10]:
# We now aim at maximizing ATP maintenance
model.objective = 'ATPM'
model.objective.expression

1.0*ATPM - 1.0*ATPM_reverse_5b752

Determine maximal ATP production rate and calculate fluxes.

In [11]:
# Solve the rates and creates a csv file to store the values
solution = model.optimize()
print('Status: ', solution.status)
print('ATP maintenance: %.2f' % solution.objective_value)
solution.fluxes.to_csv("./results/ATPmaintenanceOpt.csv")

Status:  optimal
ATP maintenance: 109.79


## Flux variability analysis

We assume that acetate and ethanol are produced by the Eut BMC.

In [12]:
tetoh_d = model.reactions.get_by_id('tetoh_d')
tetoh_d.lower_bound = 2.57

tactp_d = model.reactions.get_by_id('tactp_d')
tactp_d.lower_bound = 1.66

ACKr = model.reactions.get_by_id('ACKr')
ACKr.upper_bound = -1.66

Run FVA.

In [13]:
model.optimize()
model.summary(fva=0.99)

Metabolite,Reaction,Flux,Range,C-Number,C-Flux
ca2_e,EX_ca2_e,0.002342,[0.002342; 0.002342],0,0.00%
cl_e,EX_cl_e,0.002342,[0.002342; 0.002342],0,0.00%
cobalt2_e,EX_cobalt2_e,1.125E-05,[1.125E-05; 1.125E-05],0,0.00%
cu2_e,EX_cu2_e,0.000319,[0.000319; 0.000319],0,0.00%
etha_e,EX_etha_e,7.75,[7.75; 7.75],2,26.20%
fe2_e,EX_fe2_e,0.007227,[-0.4567; 1000],0,0.00%
glyc_e,EX_glyc_e,14.55,[14.55; 14.55],3,73.80%
k_e,EX_k_e,0.08784,[0.08784; 0.08784],0,0.00%
mg2_e,EX_mg2_e,0.003904,[0.003904; 0.003904],0,0.00%
mn2_e,EX_mn2_e,0.0003109,[0.0003109; 0.0003109],0,0.00%


## Determine the fate of acetaldehyde within the BMC and the cytosol

In [14]:
model.metabolites.acald_d.summary(fva=0.99)

Percent,Flux,Range,Reaction,Definition
100.00%,7.75,[7.75; 7.86],ETHAAL_d,etha_d --> acald_d + nh4_d
Percent,Flux,Range,Reaction,Definition
33.16%,-2.57,[-3.93; -2.57],ACALDH_d,acald_d + nad_d --> accoa_d + nadh_d
33.16%,-2.57,[-3.93; -2.57],ACALRE_d,acald_d + nadh_d --> etoh_d + nad_d
33.68%,-2.61,[-2.72; 0],tacald_d,acald_d --> acald_c


In [15]:
model.metabolites.acald_c.summary(fva=0.99)

Percent,Flux,Range,Reaction,Definition
0.00%,0,[-0.1217; 0],ACALDtpp,acald_p <=> acald_c
0.00%,0,[0; 1.36],ALCD2x,etoh_c + nad_c <=> acald_c + h_c + nadh_c
0.00%,0,[0; 0.6917],DRPA,2dr5p_c --> acald_c + g3p_c
0.00%,0,[0; 0.4612],THRA,thr__L_c --> acald_c + gly_c
0.00%,0,[0; 23.46],THRA2,athr__L_c --> acald_c + gly_c
100.00%,2.61,[0; 2.72],tacald_d,acald_d --> acald_c
Percent,Flux,Range,Reaction,Definition
100.00%,-2.61,[-26.07; -0.7574],ACALD,acald_c + coa_c + nad_c <=> accoa_c + h_c + nadh_c
0.00%,0,[-0.5476; 0],ALDD2y,acald_c + h2o_c + nadp_c --> ac_c + 2.0 h_c + nadph_c


## Determine the origin and fate of acetyl-P within cytosol

In [16]:
model.metabolites.actp_c.summary(fva=0.99)

Percent,Flux,Range,Reaction,Definition
100.00%,2.57,[2.57; 3.93],tactp_d,actp_d --> actp_c
Percent,Flux,Range,Reaction,Definition
64.59%,-1.66,[-2.755; -1.66],ACKr,ac_c + atp_c <-- actp_c + adp_c
35.41%,-0.91,[-2.27; 0.1852],PTAr,accoa_c + pi_c <=> actp_c + coa_c


## Determine the origin and fate of acetate within cytosol

In [17]:
model.metabolites.ac_c.summary(fva=0.99)

Percent,Flux,Range,Reaction,Definition
86.37%,1.66,[1.66; 2.755],ACKr,ac_c + atp_c <-- actp_c + adp_c
6.93%,0.1331,[0.1331; 0.4069],ACODA,acorn_c + h2o_c --> ac_c + orn_c
0.00%,0,[0; 998.3],ACt4pp,ac_p + na1_p --> ac_c + na1_c
0.00%,0,[0; 0.1653],AGDC,acgam6p_c + h2o_c --> ac_c + gam6p_c
0.00%,0,[0; 0.5476],ALDD2y,acald_c + h2o_c + nadp_c --> ac_c + 2.0 h_c + nadph_c
0.00%,0,[0; 0.5476],CITL,cit_c --> ac_c + oaa_c
5.79%,0.1114,[0.1114; 0.4764],CYSS,acser_c + h2s_c --> ac_c + cys__L_c + h_c
0.00%,0,[0; 0.5476],HXCT,accoa_c + hxa_c --> ac_c + hxcoa_c
0.00%,0,[0; 0.3983],POX,h2o_c + pyr_c + q8_c --> ac_c + co2_c + q8h2_c
0.91%,0.01751,[0.01751; 0.02136],UHGADA,h2o_c + u3aga_c --> ac_c + u3hga_c


## Determine the origin and fate of ammonium within cytosol

In [18]:
model.metabolites.nh4_c.summary(fva=0.99)

Percent,Flux,Range,Reaction,Definition
0.00%,0,[0; 0.05764],3AMACHYD,3amac_c + h2o_c + h_c --> msa_c + nh4_c
0.00%,0,[0; 0.3295],AACTOOR,aact_c + h2o_c + o2_c --> h2o2_c + mthgxl_c + nh4_c
0.00%,0,[0; 0.5537],ADA,adn_c + h2o_c + h_c --> ins_c + nh4_c
0.00%,0,[0; 0.5537],ADD,ade_c + h2o_c + h_c --> hxan_c + nh4_c
0.00%,0,[0; 0.09436],ALLTAMH2,alltt_c + h2o_c + 2.0 h_c --> co2_c + nh4_c + urdgci_c
0.00%,0,[0; 0.5476],ASNN,asn__L_c + h2o_c --> asp__L_c + nh4_c
0.00%,0,[0; 23.46],ASPT,asp__L_c --> fum_c + nh4_c
0.00%,0,[0; 0.3651],ATPHs,atp_c + h2o_c + h_c --> itp_c + nh4_c
0.00%,0,[0; 0.05764],CBMD,cbm_c + 2.0 h_c --> co2_c + nh4_c
0.00%,0,[0; 0.369],CSND,csn_c + h2o_c + h_c --> nh4_c + ura_c
